In [45]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from keras import Sequential
from keras.layers import Dense

In [46]:
df = pd.read_csv("../data/[CLEANED]kepler-data.csv")
df.drop(columns = ["Unnamed: 0"], inplace=True)

In [47]:
ALL_COLUMNS = df.columns
ERROR_COLUMNS = [col for col in ALL_COLUMNS if "err" in col]
EXCLUDE = ["rowid", "kepid", "kepoi_name", "koi_score", "koi_disposition", "koi_pdisposition", "koi_tce_delivname", "koi_tce_plnt_num"] + ERROR_COLUMNS
TO_USE = list(set(ALL_COLUMNS) - set(EXCLUDE))

In [48]:
subset_df = df[df["koi_disposition"] != "CANDIDATE"]

In [49]:
X = subset_df[TO_USE].values
y = subset_df["koi_disposition"].apply(lambda x: x=='CONFIRMED').astype(int).values

In [50]:
X = StandardScaler().fit_transform(X)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [52]:
model = Sequential([
                    Dense(128, activation = 'relu'),
                    Dense(256, activation = 'relu'),
                    Dense(128, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
                   ])

In [53]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [54]:
model.fit(X_train, y_train, epochs = 20)

Epoch 1/20
5551/5551 [==============================] - 1s 102us/step - loss: 0.1054 - acc: 0.9647
Epoch 2/20
5551/5551 [==============================] - 0s 45us/step - loss: 0.0554 - acc: 0.9861
Epoch 3/20
5551/5551 [==============================] - 0s 45us/step - loss: 0.0521 - acc: 0.9856
Epoch 4/20
5551/5551 [==============================] - 0s 45us/step - loss: 0.0468 - acc: 0.9852
Epoch 5/20
5551/5551 [==============================] - 0s 47us/step - loss: 0.0425 - acc: 0.9872
Epoch 6/20
5551/5551 [==============================] - 0s 46us/step - loss: 0.0388 - acc: 0.9885
Epoch 7/20
5551/5551 [==============================] - 0s 46us/step - loss: 0.0354 - acc: 0.9897
Epoch 8/20
5551/5551 [==============================] - 0s 47us/step - loss: 0.0328 - acc: 0.9897
Epoch 9/20
5551/5551 [==============================] - 0s 46us/step - loss: 0.0301 - acc: 0.9912
Epoch 10/20
5551/5551 [==============================] - 0s 45us/step - loss: 0.0291 - acc: 0.9908
Epoch 11/20
5551/5

In [55]:
pred = model.predict(X_test)
pred = pred >= 0.5

In [56]:
def performance(test, pred):
    conf_matrix = confusion_matrix(test, pred)
    f1 = f1_score(test, pred)
    report = classification_report(test, pred)
    accuracy = balanced_accuracy_score(test, pred)
    kappa = cohen_kappa_score(test, pred)
    print(f"F1 Score: {f1}")
    print(f"Kappa Score: {kappa}")
    print(f"Accuracy Score: {accuracy}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(report)

In [57]:
performance(y_test, pred)

F1 Score: 0.9645714285714286
Kappa Score: 0.9482642529410067
Accuracy Score: 0.9738404229752463
Confusion Matrix:
[[935  15]
 [ 16 422]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       950
           1       0.97      0.96      0.96       438

   micro avg       0.98      0.98      0.98      1388
   macro avg       0.97      0.97      0.97      1388
weighted avg       0.98      0.98      0.98      1388

